# 3. FE_XGBoost_GSCV
Kaggle score:

pip install jupyter notebook tqdm pillow h5py seaborn scikit-learn scikit-image xgboost

Abstract:
- date 7, 8, 9少feature的数据

## Run name

In [1]:
import time

project_name = 'TalkingdataAFD2018'
step_name = 'FE_XGBoost_GSCV'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = '%s_%s_%s' % (project_name, step_name, time_str)
print('run_name: %s' % run_name)
t0 = time.time()

run_name: TalkingdataAFD2018_FE_XGBoost_GSCV_20180506_180544


## Important params

In [2]:
feature_run_name = 'TalkingdataAFD2018_FeatureExtraction_20180501_185800'
date = 100
print('date: ', date)


is_debug = False
print('is_debug: %s' % is_debug)

# epoch = 3
# batch_size = 2000 * 10000
# skip_data_len = (epoch - 1) * batch_size
# data_len = batch_size
# print('Echo: %s, Data rows: [%s, %s]' % (epoch, skip_data_len, skip_data_len + data_len))

# epoch = 2
# batch_size = 4000 * 10000
# skip_data_len = 59633310 - batch_size
# data_len = batch_size
# print('batch_size: %s' % batch_size)
# print('epoch: %s, data rows: [%s, %s]' % (epoch, skip_data_len, skip_data_len + data_len))

# run_name = '%s_date%s%s' % (run_name, date, epoch)
run_name = '%s_date%s' % (run_name, date)

print(run_name)

if is_debug:
    test_n_rows = 1 * 10000
else:
    test_n_rows = None
#     test_n_rows = 18790469

date:  100
is_debug: False
TalkingdataAFD2018_FE_XGBoost_20180506_180300_date100


In [3]:
day_rows = {
    0: {
        'n_skiprows': 1,
        'n_rows': 1 * 1000
    },
    1: {
        'n_skiprows': 1 * 1000,
        'n_rows': 2 * 1000
    },
    6: {
        'n_skiprows': 1,
        'n_rows': 2 * 1000 # 9308568
    },
    7: {
        'n_skiprows': 1 + 9308568,
        'n_rows': 2 * 1000 # 59633310
    },
    8: {
        'n_skiprows': 1 + 9308568 + 59633310,
        'n_rows': 2 * 1000 # 62945075
    },
    9: {
        'n_skiprows': 1 + 9308568 + 59633310 + 62945075,
        'n_rows': 2 * 1000 # 53016937
    }
}
# n_skiprows = day_rows[date]['n_skiprows']
# n_rows = day_rows[date]['n_rows']

## Import PKGs

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import time
import random
import zipfile
import h5py
import pickle
import math
from PIL import Image
import shutil

from tqdm import tqdm
import multiprocessing
from multiprocessing import cpu_count

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

random_num = np.random.randint(10000)
cpu_amount = cpu_count()

print('cpu_amount: %s' % (cpu_amount - 2))
print('random_num: %s' % random_num)

cpu_amount: 30
random_num: 1252


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Project folders

In [5]:
cwd = os.getcwd()

input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

train_csv_file = os.path.join(input_folder, 'train.csv')
train_sample_csv_file = os.path.join(input_folder, 'train_sample.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_csv_file = os.path.join(input_folder, 'sample_submission.csv')

print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('train_sample_csv_file: \t\t%s' % train_sample_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)
print('sample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/input
output_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/output
model_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/model
feature_folder: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/feature
log_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/log

train_csv_file: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/input/train.csv
train_sample_csv_file: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/input/train_sample.csv
test_csv_file: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/input/test.csv
sample_submission_csv_file: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/input/sample_submission.csv


## Load feature

In [6]:
sample_submission_csv = pd.read_csv(sample_submission_csv_file)
print('sample_submission_csv.shape: \t', sample_submission_csv.shape)
display(sample_submission_csv.head(2))

print('train_csv: %.2f Mb' % (sys.getsizeof(sample_submission_csv)/1024./1024.))

sample_submission_csv.shape: 	 (18790469, 2)


,click_id,is_attributed
0,0,0
1,1,0


train_csv: 286.72 Mb


In [7]:
def save_feature(x_data, y_data, file_name):
    print(y_data[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: %s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('x_data', data=x_data)
        h.create_dataset('y_data', data=y_data)
    print('File saved:   %s' % file_name)

def load_feature(file_name):
    with h5py.File(file_name, 'r') as h:
        x_data = np.array(h['x_data'])
        y_data = np.array(h['y_data'])
    print('File loaded:  %s' % file_name)
    print(y_data[:5])
    
    return x_data, y_data


def save_test_feature(x_test, click_ids, file_name):
    print(click_ids[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: %s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('x_test', data=x_test)
        h.create_dataset('click_ids', data=click_ids)
    print('File saved:   %s' % file_name)

def load_test_feature(file_name):
    with h5py.File(file_name, 'r') as h:
        x_test = np.array(h['x_test'])
        click_ids = np.array(h['click_ids'])
    print('File loaded:  %s' % file_name)
    print(click_ids[:5])
    
    return x_test, click_ids

In [8]:
def save_feature_map(feature_map, file_name):
    print(feature_map[:5])
    feature_map_encode = []
    for item in feature_map:
        feature_name_encode = item[1].encode('UTF-8')
        feature_map_encode.append((item[0], feature_name_encode))
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('feature_map', data=feature_map_encode)
    print('File saved: \t%s' % file_name)

def load_feature_map(file_name):
    with h5py.File(file_name, 'r') as h:
        feature_map_encode = np.array(h['feature_map'])
    print('File loaded: \t%s' % file_name)
    feature_map = []
    for item in feature_map_encode:
        feature_name = item[1].decode('UTF-8')
        feature_map.append((int(item[0]), feature_name))
    print(feature_map[:5])
    
    return feature_map

In [9]:
def describe(data):
    if isinstance(data, list):
        print(len(data), '\t\t%.2f Mb' % (sys.getsizeof(data)/1024./1024.))
    else:
        print(data.shape, '\t%.2f Mb' % (sys.getsizeof(data)/1024./1024.))

test_np = np.ones((5000, 10))
describe(test_np)
describe(list(range(5000)))

(5000, 10) 	0.38 Mb
5000 		0.04 Mb


In [10]:
%%time

feature_files = []
x_train = []
y_train = []
if date == 100:
    for key in [7, 8, 9]:
        y_proba_file = os.path.join(feature_folder, 'feature_%s_date%s.p' % (feature_run_name, key))
        feature_files.append(y_proba_file)
        x_train_date, y_train_date = load_feature(y_proba_file)
        x_train.append(x_train_date)
        y_train.append(y_train_date)
    x_train = np.concatenate(x_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
else:
    y_proba_file = os.path.join(feature_folder, 'feature_%s_date%s.p' % (feature_run_name, date))
    feature_files.append(y_proba_file)
    x_train, y_train = load_feature(y_proba_file)

# Use date 6 as validation dataset
y_proba_file = os.path.join(feature_folder, 'feature_%s_date%s.p' % (feature_run_name, 6))
feature_files.append(y_proba_file)
x_val, y_val = load_feature(y_proba_file)


print('*' * 80)
describe(x_train)
describe(y_train)
describe(x_val)
describe(y_val)

File loaded:  /data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_20180501_185800_date7.p
[0 0 0 0 0]
File loaded:  /data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_20180501_185800_date8.p
[0 0 0 0 0]
File loaded:  /data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_20180501_185800_date9.p
[0 0 0 0 0]
File loaded:  /data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_20180501_185800_date6.p
[0 0 0 0 0]
********************************************************************************
(175595322, 34) 	45549.32 Mb
(175595322,) 	167.46 Mb
(9308568, 34) 	2414.64 Mb
(9308568,) 	8.88 Mb
CPU times: user 16.2 s, sys: 40.4 s, total: 56.6 s
Wall time: 56.5 s


In [11]:
%%time

# from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# x_train, x_val, y_train, y_val = train_test_split(x_data[skip_data_len: data_len], y_data[skip_data_len: data_len], test_size=0.1, random_state=random_num, shuffle=True)
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=random_num, shuffle=False)

# x_train = x_train[skip_data_len: skip_data_len + data_len]
# y_train = y_train[skip_data_len: skip_data_len + data_len]

x_train, y_train = shuffle(x_train, y_train, random_state=random_num)
describe(x_train)
describe(y_train)
describe(x_val)
describe(y_val)

(175595322, 34) 	45549.32 Mb
(175595322,) 	167.46 Mb
(9308568, 34) 	2414.64 Mb
(9308568,) 	8.88 Mb
CPU times: user 36.6 s, sys: 10.5 s, total: 47.1 s
Wall time: 47.1 s


In [12]:
gc.collect()

0

## Train

In [15]:
# %%time
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


clf = xgb.XGBClassifier(
    max_depth=20, 
    learning_rate=0.1, 
    n_estimators=5000, 
    silent=False, 
    objective='gpu:binary:logistic', 
    booster='gbtree', 
#     n_jobs=1, 
    nthread=None, 
    gamma=0, 
    min_child_weight=1, 
    max_delta_step=0, 
    subsample=0.5, 
    colsample_bytree=0.7, 
    colsample_bylevel=0.7, 
    reg_alpha=0, 
    reg_lambda=1, 
    scale_pos_weight=97, 
    base_score=0.5, 
    random_state=0, 
    seed=None, 
    missing=None,
    # booster params
    num_boost_round=50,
    early_stopping_rounds=30,
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    eval_metric=['auc'],

)

parameters = {
    'reg_alpha':[0.3], 
    'reg_lambda':[0.8],
    'scale_pos_weight': [1, 98, 200]
}

grid_search = GridSearchCV(clf, parameters, verbose=2, cv=3)
grid_search.fit(x_train, y_train)


In [ ]:
print('*' * 80)
y_train_proba = grid_search.predict_proba(x_train)
print(y_train_proba.shape)
print(y_train_proba[:10])
y_train_pred = (y_train_proba[:, 1]>=0.5).astype(int)
acc_train = accuracy_score(y_train, y_train_pred)
roc_train = roc_auc_score(y_train, y_train_proba[:, 1])
print('acc_train: %.4f \t roc_train: %.4f' % (acc_train, roc_train))

# y_train_pred = grid_search.predict(x_train)
# acc_train = accuracy_score(y_train, y_train_pred)
# roc_train = roc_auc_score(y_train, y_train_proba[:, 1])
# print('acc_train: %.4f \t roc_train: %.4f' % (acc_train, roc_train))

y_val_proba = grid_search.predict_proba(x_val)
print(y_val_proba.shape)
print(y_val_proba[:10])
y_val_pred = (y_val_proba[:, 0]>=0.5).astype(int)
acc_val = accuracy_score(y_val, y_val_pred)
roc_val = roc_auc_score(y_val, y_val_proba[:, 1])
print('acc_val:   %.4f \t roc_val:   %.4f' % (acc_val, roc_val))

In [ ]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

## Feature importance

In [ ]:
xgboost_model = best_estimator_.get_boostor()

In [ ]:
from xgboost import plot_importance

fig, ax = plt.subplots(figsize=(10,int(x_train.shape[1]/2)))
plot_importance(xgboost_model, height=0.5, ax=ax, max_num_features=300)
plt.show()

In [ ]:
feature_map_file_name = os.path.join(feature_folder, 'feature_map_TalkingdataAFD2018_FeatureExtraction_test_20180501_143516.p')

feature_map = load_feature_map(feature_map_file_name)
print(len(feature_map))
print(feature_map[:5])

feature_dict = {}
for item in feature_map:
    feature_dict[item[0]] = item[1]
print(list(feature_dict.keys())[:5])
print(list(feature_dict.values())[:5])

In [ ]:
# print(dir(grid_search.best_estimator_.get_booster()))
importance_score = xgboost_model.get_fscore()
sorted_score = []
for key in importance_score:
    indx = int(key[1:])
    sorted_score.append((importance_score[key], key, indx, feature_dict[indx]))
dtype = [('importance_score', int), ('key', 'S50'), ('indx', int), ('name', 'S50')]
importance_table = np.array(sorted_score, dtype=dtype)
display(importance_table[:2])
importance_table = np.sort(importance_table, axis=0, order=['importance_score'])
importance_table = importance_table[::-1]
display(importance_table)

In [ ]:
# del x_train
# del y_train
gc.collect()

## Predict

In [ ]:
run_name_acc = run_name + '_' + str(int(roc_val*10000)).zfill(4)
print(run_name_acc)

In [ ]:
%%time
y_proba_file = os.path.join(feature_folder, 'feature_%s_test.p' % feature_run_name)
feature_files.append(y_proba_file)
x_test, click_ids = load_test_feature(y_proba_file)

describe(x_test)
describe(click_ids)

In [ ]:
print(feature_files)

In [ ]:
y_test_proba = grid_search.predict_proba(x_test)

print(y_test_proba.shape)
print(y_test_proba[:20])

In [ ]:
def save_proba(y_val_proba, y_val, y_test_proba, click_ids, file_name):
    print(click_ids[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: %s' % file_name)
    with h5py.File(file_name) as h:
#         h.create_dataset('y_train_proba', data=y_train_proba)
#         h.create_dataset('y_train', data=y_train)
        h.create_dataset('y_val_proba', data=y_val_proba)
        h.create_dataset('y_val', data=y_val)
        h.create_dataset('y_test_proba', data=y_test_proba)
        h.create_dataset('click_ids', data=click_ids)
    print('File saved:   %s' % file_name)

def load_proba(file_name):
    with h5py.File(file_name, 'r') as h:
#         y_train_proba = np.array(h['y_train_proba'])
#         y_train = np.array(h['y_train'])
        y_val_proba = np.array(h['y_val_proba'])
        y_val = np.array(h['y_val'])
        y_test_proba = np.array(h['y_test_proba'])
        click_ids = np.array(h['click_ids'])
    print('File loaded:  %s' % file_name)
    print(click_ids[:5])
    
    return y_val_proba, y_val, y_test_proba, click_ids


y_proba_file = os.path.join(model_folder, 'proba_%s.p' % run_name_acc)
print(y_proba_file)
save_proba(
#     y_train_proba, 
#     y_train, 
    y_val_proba, 
    y_val, 
    y_test_proba, 
    np.array(sample_submission_csv['click_id']), 
    y_proba_file
)
y_val_proba_true, y_val, y_test_proba_true, click_ids = load_proba(y_proba_file)

# print(y_train_proba_true.shape)
# print(y_train.shape)
print(y_val_proba_true.shape)
print(y_val.shape)
print(y_test_proba_true.shape)
print(len(click_ids))

In [ ]:
# %%time
submission_csv_file = os.path.join(output_folder, 'pred_%s.csv' % run_name_acc)
print(submission_csv_file)
submission_csv = pd.DataFrame({ 'click_id': click_ids , 'is_attributed': y_test_proba_true })
submission_csv.to_csv(submission_csv_file, index = False)
display(submission_csv.head())

In [ ]:
print('Time cost: %.2f s' % (time.time() - t0))

print('random_num: ', random_num)
print('date: ', date)
print(run_name_acc)
print('Done!')